# Logistic Regression

In [ ]:
import pandas as pd

# Path to data file
parquet_file_path = '../../Costa_Rica_Data/Data Acquisition Output/extracted_gee_data/en_clean_gee_data.parquet'

# Load into a DataFrame
df = pd.read_parquet(parquet_file_path)

# Display head
df.head()

,plotid,sampleid,Use,CoverType,Vegetations,Herbaceous,GrasslandShrub,CropsType,WetlandArea,LandType,...,mTPI,ndvi,ocs_1mMed,sand_1mMed,savi,silt_1mMed,slope,topDiv,wetness,Forest_Presence
9,2902,11605,Wetlands,Vegetation,Trees,Not_Applicable,Not_Applicable,Not_Applicable,Swamp (Marsh),Not_Applicable,...,8129.0,0.799771,68.0,332.15,0.393728,298.85,0.927410,1323.685053,-0.006312,Present
12,2902,11608,Grasslands,Vegetation,Herbaceous plants,Grasses,Mixed Pasture (70-90%),Not_Applicable,Not_Applicable,Not_Applicable,...,8129.0,0.796553,68.0,332.15,0.423713,298.85,2.935819,1323.685053,-0.014932,Present
13,2902,11609,Wetlands,Vegetation,Herbaceous plants,Grasses,Not_Applicable,Not_Applicable,Swamp (Marsh),Not_Applicable,...,8129.0,0.644415,65.0,340.45,0.380250,304.70,0.944368,1323.685053,-0.001697,Present
15,2902,11611,Grasslands,Vegetation,Herbaceous plants,Grasses,Mixed Pasture (70-90%),Not_Applicable,Not_Applicable,Not_Applicable,...,8129.0,0.784331,68.0,332.15,0.512687,298.85,0.927410,1323.685053,-0.027987,Present
16,2902,11612,Wetlands,Vegetation,Trees,Not_Applicable,Not_Applicable,Not_Applicable,Swamp (Marsh),Not_Applicable,...,8129.0,0.769357,65.0,340.45,0.360428,304.70,2.645556,1323.685053,-0.009912,Present


In [24]:
y = df['Use']  # Set the target variable
X = df[['ndvi','evi','savi','brightness','wetness','slope','aspect','mTPI','ocs_1mMed', 'sand_1mMed','topDiv']]  # Predictor features

# Check the result
print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")
X.head()

X shape: (91917, 11)
y shape: (91917,)


,ndvi,evi,savi,brightness,wetness,slope,aspect,mTPI,ocs_1mMed,sand_1mMed,topDiv
9,0.799771,0.389640,0.393728,0.213948,-0.006312,0.927410,180.000000,8129.0,68.0,332.15,1323.685053
12,0.796553,0.426831,0.423713,0.245222,-0.014932,2.935819,341.251312,8129.0,68.0,332.15,1323.685053
13,0.644415,0.414457,0.380250,0.291687,-0.001697,0.944368,90.000000,8129.0,65.0,340.45,1323.685053
15,0.784331,0.532381,0.512687,0.348042,-0.027987,0.927410,0.000000,8129.0,68.0,332.15,1323.685053
16,0.769357,0.346814,0.360428,0.198905,-0.009912,2.645556,314.480835,8129.0,65.0,340.45,1323.685053


In [25]:
# List columns with missing values and the count of missing values
missing_values = X.isnull().sum()

missing_values[missing_values > 0]

Series([], dtype: int64)

Linear Regression and SVM do not accept missing values encoded as NaN

In [ ]:
from sklearn.impute import SimpleImputer

# Make copy of the DataFrame to avoid SettingWithCopyWarning
X = X.copy()

# Columns to impute with the mean
columns_to_impute = ['brightness', 'savi', 'topDiv', 'wetness',
                     'mTPI', 'ndvi', 'evi']

# Initialize the imputer to fill missing values with the mean
imputer = SimpleImputer(strategy='mean')

# Apply the imputer to the selected columns using .loc
X.loc[:, columns_to_impute] = imputer.fit_transform(X[columns_to_impute])

# Ensure y corresponds to the new X after any row drop
y = y.loc[X.index]

In [27]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=30, test_size=0.2)

In [28]:
# List columns with missing values and the count of missing values
missing_values = X.isnull().sum()

missing_values[missing_values > 0]

Series([], dtype: int64)

In [29]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(X_train, y_train)

/home/sam/miniconda3/envs/mlenv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [30]:
y_prediction = model.predict(X_test)

In [31]:
model.score(X_test, y_test)

0.567286771105309

In [32]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

# Standardize data
scaler = StandardScaler()
X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)
X_test_scaled = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

# Fit model
model = LogisticRegression(max_iter=5000, solver='lbfgs')
model.fit(X_train_scaled, y_train)

y_pred = model.predict(X_test_scaled)

In [33]:
model.score(X_test, y_test)

0.0015774586597040905